In [1]:
import cv2
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
import numpy as np
import time
import os

In [34]:
# Load your trained PyTorch model
model = torch.load('V4/vit_model.pth', map_location=torch.device('cpu'))
model.eval()

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dictionary for class mapping
class_names = {
    0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G",
    7: "H", 8: "I", 9: "J", 10: "K", 11: "L", 12: "M", 13: "N",
    14: "O", 15: "P", 16: "Q", 17: "R", 18: "S", 19: "T", 20: "U",
    21: "V", 22: "W", 23: "X", 24: "Y", 25: "Z"
}

# Output directory for saving frames
actual_class = "E"
output_dir = "testing_" + actual_class
os.makedirs(output_dir, exist_ok=True)

frame_count = 0

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_dir + '.avi', fourcc, 5.0, (640, 480))

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = transform(img).unsqueeze(0)

    with torch.no_grad():
        output = model(img)
    _, predicted_class = torch.max(output, 1)
    confidence = torch.nn.functional.softmax(output, dim=1)[0][predicted_class].item()
    
    # Get the class name from the mapping
    predict_class = class_names.get(predicted_class.item())
    
    class_text = f"Class: {predict_class}"
    confidence_text = f"Confidence: {confidence:.2f}"
    
    # Display text
    cv2.putText(frame, class_text, (10, 175), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 153), 3)
    cv2.putText(frame, confidence_text, (10, 225), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 153), 3)
    
    # Simpan frame yang telah diprediksi ke dalam file video
    out.write(frame)
    cv2.imshow('BISINDO Real-Time Classification', frame)
    
    # Simpan frame dalam file gambar
    frame_filename = os.path.join(output_dir, f'frame_{frame_count:04d}_{predict_class}.png')
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6512\3582699760.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('V4/vit_model.pth', map_location=torch.de